In [ ]:
!pip install langchain langchain-openai langchain-core langchain_community docx2txt pypdf langchain_chroma

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
import os
os.environ["GOOGLE_API_KEY"] = "your_google_api_key"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "your_langchain_api_key"
os.environ["LANGSMITH_ENDPOINT"] ="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "langchain-course"

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Initialize the Gemini model
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",    # Gemini 1.5 Flash (latest fast model)
    google_api_key="your_google_api_key",  # <-- Set your Gemini API key here
    temperature=0.7              # Optional: control randomness
)

# Send a prompt
llm_response = llm.invoke("Tell me about LangChain!")

# Print the response
print(llm_response.content)


In [ ]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

In [ ]:
chain = llm | output_parser
chain.invoke("Tell me about Langchain!")

'LangChain is an open-source framework designed to facilitate the development of applications that leverage large language models (LLMs). It provides a structured interface to integrate these models with various data sources, making it easier for developers to build complex applications that utilize natural language processing (NLP) capabilities. Here are some key aspects of LangChain:\n\n1. **Modularity**: LangChain is built around a modular architecture, where various components can be combined and reused. These components include models, data connectors, prompt templates, memory, and more.\n\n2. **Integration with LLMs**: LangChain is designed to work seamlessly with various large language models, including those from OpenAI, Hugging Face, and other providers. This flexibility allows developers to choose the model that best fits their use case.\n\n3. **Data Connections**: The framework supports integration with various types of data sources, such as databases, document stores, APIs,

In [ ]:
    #generate a controlled bot response so that we have a structured and controlled response type
    from typing import List
    from pydantic import BaseModel, Field

    class MobileReview(BaseModel):
        phone_model: str = Field(description="Name and model of the phone")
        rating: float = Field(description="Overall rating out of 5")
        pros: List[str] = Field(description="List of positive aspects")
        cons: List[str] = Field(description="List of negative aspects")
        summary: str = Field(description="Brief summary of the review")

    review_text = """
    Just got my hands on the new Galaxy S21 and wow, this thing is slick! The screen is gorgeous,
    colors pop like crazy. Camera's insane too, especially at night - my Insta game's never been
    stronger. Battery life's solid, lasts me all day no problem.
    Not gonna lie though, it's pretty pricey. And what's with ditching the charger? C'mon Samsung.
    Also, still getting used to the new button layout, keep hitting Bixby by mistake.
    Overall, I'd say it's a solid 4 out of 5. Great phone, but a few annoying quirks keep it from
    being perfect. If you're due for an upgrade, definitely worth checking out!
    """

    structured_llm = llm.with_structured_output(MobileReview)
    output = structured_llm.invoke(review_text)
    print(output)
    print(output.pros)


phone_model='Samsung Galaxy S21' rating=4.0 pros=['Gorgeous display', 'Vibrant colors', 'Impressive camera performance, especially in low light', 'Good battery life'] cons=['High price point', 'No charger included in the box', 'New button layout takes time to get used to'] summary='Overall, a great phone with exceptional features, though pricey and a few quirks.'
['Gorgeous display', 'Vibrant colors', 'Impressive camera performance, especially in low light', 'Good battery life']


In [ ]:
    from typing import List
    from pydantic import BaseModel, Field

    class MobileReview(BaseModel):
        keyboard_model: str = Field(description="Name and model of the phone")
        rating: float = Field(description="Overall rating out of 5")
        pros: List[str] = Field(description="List of positive aspects")
        cons: List[str] = Field(description="List of negative aspects")
        summary: str = Field(description="Brief summary of the review")

    review_text = """
    Just picked up the Keychron K6, and I gotta say, this thing is a game-changer! The build feels super solid, and the compact 65% layout is perfect—small enough to save desk space but still has arrow keys (thank god). \
    Typing on it is insanely satisfying, especially with the Gateron Brown switches—nice tactile feedback without being obnoxiously loud. Also, hot-swappable keys? Yes, please.The Bluetooth connectivity is smooth,
    and I love that I can switch between my laptop, iPad, and PC with just a button press. Battery life is pretty great too—I’ve been using it for days without needing a charge. Oh, and the RGB lighting is a nice touch; not too flashy,
    but just enough to give my setup some personality.A few small gripes, though. The stock keycaps are decent, but they’re ABS plastic, so they’ll probably get shiny over time. Also, no dedicated function row, so if you use F keys a lot,
    you’ll need to get used to the Fn combos.Overall, I’m really loving this keyboard. It’s compact, versatile, and just a joy to type on. If you want a wireless mechanical keyboard that doesn’t suck, the Keychron K6 is absolutely worth checking out. Solid 4.7/5 from me!
    """

    structured_llm = llm.with_structured_output(MobileReview)
    output = structured_llm.invoke(review_text)
    print(output)
    print(output.pros)


keyboard_model='Keychron K6' rating=4.7 pros=['Solid build quality', 'Compact 65% layout', 'Satisfying typing experience', 'Gateron Brown switches', 'Hot-swappable keys', 'Smooth Bluetooth connectivity', 'Excellent battery life', 'Nice RGB lighting'] cons=['Stock keycaps are ABS plastic', 'No dedicated function row'] summary='The Keychron K6 is a compact and versatile mechanical keyboard that offers a satisfying typing experience with its Gateron Brown switches and solid build quality. While it has a few minor drawbacks, it excels in battery life and connectivity, making it a great choice for anyone looking for a wireless mechanical keyboard.'
['Solid build quality', 'Compact 65% layout', 'Satisfying typing experience', 'Gateron Brown switches', 'Hot-swappable keys', 'Smooth Bluetooth connectivity', 'Excellent battery life', 'Nice RGB lighting']


#Prompt Template

In [ ]:
    from langchain_core.prompts import ChatPromptTemplate

    prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")
    #chain = prompt | llm | output_parser
    result = prompt.invoke({"topic": "airpods"})
    print(result)

messages=[HumanMessage(content='Tell me a short joke about airpods', additional_kwargs={}, response_metadata={})]


In [ ]:
chain = prompt | llm | output_parser
chain.invoke({"topic": "notebook"})

'Why did the notebook break up with the pencil?\n\nBecause it found someone more write for it!'

# Putting it all together in one cell

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

#define the prompt
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")

#Initialize the LLM
llm = ChatOpenAI(model="gpt-4o-mini")

#Define the output parser
output_parser = StrOutputParser()

#Compose the chain
chain = prompt | llm | output_parser

#Use the chain
result = chain.invoke({"topic": "Machine Learning"})
print(result)

Why did the neural network break up with the dataset?

It had too many features but lacked good training!


In [ ]:
#Messages passed in "LIST" that defines the system message as what it will be behaving like in the conversation

from langchain_core.messages import HumanMessage, SystemMessage
messages = [
        SystemMessage(content="You are a helpful assistant that tells jokes."),
        HumanMessage(content="Tell me about programming")
    ]
response = llm.invoke(messages)
print(response)

content='Sure! Programming is like writing a recipe for a computer. You give it a list of instructions to follow, and if everything is done correctly, it can make amazing things happen! \n\nAnd here’s a little joke for you:\n\nWhy do programmers prefer dark mode?\n\nBecause light attracts bugs!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 24, 'total_tokens': 83, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b8bc95a0ac', 'id': 'chatcmpl-BEKREe2aX2WnmYugcS39CYC8xONP6', 'finish_reason': 'stop', 'logprobs': None} id='run-fba08d5a-df65-4417-a94c-9fcd279f4c5f-0' usage_metadata={'input_tokens': 24, 'output_tokens': 59, 'total_tokens': 83, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_tok

In [ ]:
# Messages passed in Tuple

template = ChatPromptTemplate([
        ("system", "You are a helpful assistant that tells jokes."),
        ("human", "Tell me about {topic}")
    ])
chain = template | llm
response = chain.invoke({"topic": "programming"})
print(response)


content='Sure! Here’s a light-hearted joke about programming:\n\nWhy do programmers prefer dark mode?\n\nBecause light attracts bugs! \n\nIf you want to hear more programming jokes or need information about a specific programming topic, just let me know!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 24, 'total_tokens': 71, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b8bc95a0ac', 'id': 'chatcmpl-BEKRH668RQ8Q2Bbjylf1LKGOozn6q', 'finish_reason': 'stop', 'logprobs': None} id='run-17b8bf85-8e2b-42f3-b50c-457561122d26-0' usage_metadata={'input_tokens': 24, 'output_tokens': 47, 'total_tokens': 71, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}

In [ ]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
import os

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len
)

#load and split the document
docs_loader = Docx2txtLoader("/content/kbs/AI Diplomats in Martian Colonies.docx")
documents = docs_loader.load()
print(len(documents))
splits = text_splitter.split_documents(documents)
print(f"Split the documents into {len(splits)} chunks.")

1
Split the documents into 17 chunks.


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len
)

splits = text_splitter.split_documents(documents)
print(f"Split the documents into {len(splits)} chunks.")


Split the documents into 17 chunks.


In [ ]:
print(documents[0])

page_content='Title: Protocols of Peace: The Role of AI Diplomats in Inter-Colony Relations on Mars

Author: Dr. Imani Okeke
Date: May 2026
Institution: Martian Institute for Post-Terrestrial Governance (MIPTG)

By 2035, with five autonomous colonies established on Mars, governance challenges began to emerge around shared resources, territorial claims, and scientific collaboration. To address the communication gaps and human bias, the Martian Council sanctioned the deployment of AI diplomats programmed with advanced natural language processing, legal logic, and inter-colony ethics modules.

The AI diplomats—nicknamed “Red Envoys”—mediated disputes, drafted treaty frameworks, and optimized trade agreements involving water extraction, energy allocation, and Martian orbital satellite access. Each AI envoy was trained on 200 years of Earth diplomatic history and continuously updated via shared interplanetary networks.

Case studies from Olympus Mons Colony and Valles Marineris Habitat show

In [ ]:
print(splits[0])

page_content='Title: Protocols of Peace: The Role of AI Diplomats in Inter-Colony Relations on Mars' metadata={'source': '/content/kbs/AI Diplomats in Martian Colonies.docx'}


In [ ]:
print(splits[0].metadata)
print(splits[0].page_content)

{'source': '/content/kbs/AI Diplomats in Martian Colonies.docx'}
Title: Protocols of Peace: The Role of AI Diplomats in Inter-Colony Relations on Mars


In [ ]:
def load_documents(folder_path: str) -> List[Document]:
    documents = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        print (file_path)
        if filename.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif filename.endswith('.docx'):
            loader = Docx2txtLoader(file_path)
        else:
            print(f"Unsupported file type: {filename}")
            continue
        documents.extend(loader.load())   #extract content from the files and save in documents
    return documents

folder_path = "/content/kbs"
documents = load_documents(folder_path)

splits = text_splitter.split_documents(documents)
print(f"Splitted the documents into {len(splits)} chunks.")

/content/kbs/Urban Rewilding Case Study.docx
/content/kbs/Neo-Feudalism in the Age of Smart Contracts.docx
/content/kbs/Terraforming Coral Reefs for Atmospheric Stabilization.docx
/content/kbs/AI Diplomats in Martian Colonies.docx
Splitted the documents into 74 chunks.


In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
document_embeddings = embeddings.embed_documents([split.page_content for split in splits])
print(f"Created embeddings for {len(document_embeddings)} document chunks.")
print(document_embeddings[0][:5])  # Printing first 5 elements of the first embedding

Created embeddings for 74 document chunks.
[0.005381679628044367, -0.0017905021086335182, -0.010431031696498394, 0.012262227013707161, -0.007826664485037327]


In [ ]:
!pip install numpy

In [ ]:
!pip install --upgrade numpy

  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-chroma 0.2.2 requires numpy<2.0.0,>=1.22.4; python_version < "3.12", but you have numpy 2.2.4 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.4 which is incompatible.


In [ ]:
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
document_embeddings = embedding_function.embed_documents([split.page_content for split in splits])
print(f"Created embeddings for {len(document_embeddings)} document chunks.")
print(document_embeddings[0][:5])  # Printing first 5 elements of the first embedding


<ipython-input-31-832ff12651fc>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or dataset

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Created embeddings for 74 document chunks.
[0.028334224596619606, 0.03767409548163414, 0.09719788283109665, 0.014158791862428188, 0.05144160985946655]


In [ ]:
from langchain_chroma import Chroma

collection_name = "my_collection"
vectorstore = Chroma.from_documents(
    collection_name=collection_name,
    documents=splits,
    embedding=embedding_function,
    persist_directory="./chroma_db"
)
print("Vector store created and persisted to './chroma_db'")


Vector store created and persisted to './chroma_db'


In [ ]:
#perform similarity search
query = "What is CoralTerraform and who founded it?"
search_results= vectorstore.similarity_search(query, k=2)
print(f"\nTop 1 most relevant chunks for the query: '{query}' are:\n\n")
for i, result in enumerate(search_results , 1):
    print(f"Results {i}:")
    print(f"Source: {result.metadata.get('source', 'Unknown')}")
    print(f"Content: {result.page_content}")
    print()



Top 1 most relevant chunks for the query: 'What is CoralTerraform and who founded it?' are:


Results 1:
Source: /content/kbs/Terraforming Coral Reefs for Atmospheric Stabilization.docx
Content: In response to accelerating climate change, PORA launched the CoralTerraform Initiative, a

Results 2:
Source: /content/kbs/Terraforming Coral Reefs for Atmospheric Stabilization.docx
Content: Pilot reefs installed off the coast of Palau were seeded with synthetic coral larvae developed in



In [ ]:
#making retriever to pass the retrieved vector store in the chain
retriever = vectorstore.as_retriever(search_kwargs={"k":2})
retriever.invoke("What is CoralTerraform and who founded it?")

[Document(id='9fec36d9-5b01-4c95-a1f7-b14e395f42be', metadata={'source': '/content/kbs/Terraforming Coral Reefs for Atmospheric Stabilization.docx'}, page_content='In response to accelerating climate change, PORA launched the CoralTerraform Initiative, a'),
 Document(id='cbcbafeb-250b-4d87-92e2-59fe193afbe5', metadata={'source': '/content/kbs/Terraforming Coral Reefs for Atmospheric Stabilization.docx'}, page_content='Pilot reefs installed off the coast of Palau were seeded with synthetic coral larvae developed in')]

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

template = """ Answer the question based only on the following context:{context}
Question: {question}

Answer: """
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
rag_chain = (
   { "context": retriever, "question": RunnablePassthrough() } | prompt
)
rag_chain.invoke("What is CoralTerraform and who founded it?")

ChatPromptValue(messages=[HumanMessage(content=" Answer the question based only on the following context:[Document(id='9fec36d9-5b01-4c95-a1f7-b14e395f42be', metadata={'source': '/content/kbs/Terraforming Coral Reefs for Atmospheric Stabilization.docx'}, page_content='In response to accelerating climate change, PORA launched the CoralTerraform Initiative, a'), Document(id='cbcbafeb-250b-4d87-92e2-59fe193afbe5', metadata={'source': '/content/kbs/Terraforming Coral Reefs for Atmospheric Stabilization.docx'}, page_content='Pilot reefs installed off the coast of Palau were seeded with synthetic coral larvae developed in')]\nQuestion: What is CoralTerraform and who founded it?\n\nAnswer: ", additional_kwargs={}, response_metadata={})])

In [ ]:
def doc2str(docs):
  return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
rag_chain = (
    {"context": retriever | doc2str, "question": RunnablePassthrough()} | prompt
)
rag_chain.invoke("What is CoralTerraform and who founded it?")


ChatPromptValue(messages=[HumanMessage(content=' Answer the question based only on the following context:In response to accelerating climate change, PORA launched the CoralTerraform Initiative, a\n\nPilot reefs installed off the coast of Palau were seeded with synthetic coral larvae developed in\nQuestion: What is CoralTerraform and who founded it?\n\nAnswer: ', additional_kwargs={}, response_metadata={})])

In [ ]:
rag_chain =(
    {"context": retriever | doc2str, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
question = "What is CoralTerraform and who founded it?"
response = rag_chain.invoke(question)
print(response)

CoralTerraform is the CoralTerraform Initiative launched by PORA in response to accelerating climate change.


#Conversation RAG

Handeling followup question

In [ ]:
#Example Conrsation
from langchain_core.messages import HumanMessage, AIMessage

chat_history = []
chat_history.extend([
    HumanMessage(content=question),
    AIMessage(content=response)
                ])


In [ ]:
chat_history

[HumanMessage(content='What is CoralTerraform and who founded it?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='CoralTerraform is the CoralTerraform Initiative launched by PORA in response to accelerating climate change.', additional_kwargs={}, response_metadata={})]

In [ ]:
from langchain_core.prompts import MessagesPlaceholder
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question"
    " which might reference context in the chat history"
    " formulate a standline question which can be understood"
    " without thr chat history, do not answer the question"
    " just frefomulate it if needed and otherwise resturn it as it is"
)
contextualize_q_system_prompt=ChatPromptTemplate.from_messages(
    [
      ("system", contextualize_q_system_prompt),
      MessagesPlaceholder(variable_name="chat_history"),
      ("human", "{input}")
    ]
)
contextualize_chain = contextualize_q_system_prompt | llm | StrOutputParser()
contextualize_chain.invoke({"input": "Tell me more about it", "chat_history": chat_history})

'Can you provide more details about CoralTerraform and its objectives?'

In [ ]:
from langchain.chains import create_history_aware_retriever
history_aware_retriever = create_history_aware_retriever(
     llm, retriever, contextualize_q_system_prompt
 )
history_aware_retriever.invoke({"input": "Tell me more about it", "chat_history": chat_history})

[Document(id='9fec36d9-5b01-4c95-a1f7-b14e395f42be', metadata={'source': '/content/kbs/Terraforming Coral Reefs for Atmospheric Stabilization.docx'}, page_content='In response to accelerating climate change, PORA launched the CoralTerraform Initiative, a'),
 Document(id='6a6278b3-22f7-47da-a177-9c634dd7c401', metadata={'source': '/content/kbs/Terraforming Coral Reefs for Atmospheric Stabilization.docx'}, page_content='Initiative, a fictional program aiming to bioengineer coral reefs to sequester carbon dioxide at a')]

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the following context to answer the user's question."),
    ("system", "Context: {context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_system_prompt
)

question_anwser_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_anwser_chain)

In [ ]:
response = rag_chain.invoke({"input": question, "chat_history": chat_history})
print(response)

{'input': 'What is CoralTerraform and who founded it?', 'chat_history': [HumanMessage(content='What is CoralTerraform and who founded it?', additional_kwargs={}, response_metadata={}), AIMessage(content='CoralTerraform is the CoralTerraform Initiative launched by PORA in response to accelerating climate change.', additional_kwargs={}, response_metadata={})], 'context': [Document(id='9fec36d9-5b01-4c95-a1f7-b14e395f42be', metadata={'source': '/content/kbs/Terraforming Coral Reefs for Atmospheric Stabilization.docx'}, page_content='In response to accelerating climate change, PORA launched the CoralTerraform Initiative, a'), Document(id='cbcbafeb-250b-4d87-92e2-59fe193afbe5', metadata={'source': '/content/kbs/Terraforming Coral Reefs for Atmospheric Stabilization.docx'}, page_content='Pilot reefs installed off the coast of Palau were seeded with synthetic coral larvae developed in')], 'answer': 'CoralTerraform is an initiative aimed at addressing the challenges posed by climate change, sp

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = []
question1 = "What is CoralTerraform and who founded it?"
answer1 = rag_chain.invoke({"input": question1, "chat_history": chat_history})['answer']
chat_history.extend([
    HumanMessage(content=question1),
    AIMessage(content=answer1)
])

print(f"Human: {question1}")
print(f"AI: {answer1}\n")

question2 = "Tell me more about it"
answer2 = rag_chain.invoke({"input": question2, "chat_history": chat_history})['answer']
chat_history.extend([
    HumanMessage(content=question2),
    AIMessage(content=answer2)
])

print(f"Human: {question2}")
print(f"AI: {answer2}")


Human: What is CoralTerraform and who founded it?
AI: CoralTerraform is an initiative launched by PORA (Preservation of Resources and Aquatic life) in response to the accelerating impacts of climate change on coral reefs. The initiative focuses on creating artificial reefs using synthetic coral larvae to support coral restoration efforts.

Human: Tell me more about it
AI: The CoralTerraform Initiative aims to address the challenges posed by climate change to coral reefs, which are vital ecosystems that support marine biodiversity and act as natural barriers against coastal erosion. 

Key aspects of the CoralTerraform Initiative include:

1. **Bioengineering**: The initiative involves the development of bioengineered coral species that are more resilient to warming ocean temperatures and ocean acidification. This could enhance their chances of survival and growth in changing environmental conditions.

2. **Carbon Sequestration**: One of the goals of CoralTerraform is to enhance the abil

#Multiple User Chatbot


In [ ]:
import sqlite3
from datetime import datetime
import uuid

DB_NAME = "rag_app.db"

def get_db_connection():
    conn = sqlite3.connect(DB_NAME)
    conn.row_factory = sqlite3.Row
    return conn

def create_application_logs():
    conn = get_db_connection()
    conn.execute('''CREATE TABLE IF NOT EXISTS application_logs
    (id INTEGER PRIMARY KEY AUTOINCREMENT,
    session_id TEXT,
    user_query TEXT,
    gpt_response TEXT,
    model TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP)''')
    conn.close()

def insert_application_logs(session_id, user_query, gpt_response, model):
    conn = get_db_connection()
    conn.execute('INSERT INTO application_logs (session_id, user_query, gpt_response, model) VALUES (?, ?, ?, ?)',
                 (session_id, user_query, gpt_response, model))
    conn.commit()
    conn.close()

def get_chat_history(session_id):
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute('SELECT user_query, gpt_response FROM application_logs WHERE session_id = ? ORDER BY created_at', (session_id,))
    messages = []
    for row in cursor.fetchall():
        messages.extend([
            {"role": "human", "content": row['user_query']},
            {"role": "ai", "content": row['gpt_response']}
        ])
    conn.close()
    return messages

# Initialize the database
create_application_logs()


In [ ]:
# Example usage for a new user
session_id = str(uuid.uuid4())
question = "What is CoralTerraform and who founded it?"
chat_history = get_chat_history(session_id)
answer = rag_chain.invoke({"input": question, "chat_history": chat_history})['answer']
insert_application_logs(session_id, question, answer, "gpt-3.5-turbo")
print(f"Human: {question}")
print(f"AI: {answer}\n")

# Example of a follow-up question
question2 = "Tell me more about it"
chat_history = get_chat_history(session_id)
answer2 = rag_chain.invoke({"input": question2, "chat_history": chat_history})['answer']
insert_application_logs(session_id, question2, answer2, "gpt-3.5-turbo")
print(f"Human: {question2}")
print(f"AI: {answer2}")

print (chat_history)


[]
Human: What is CoralTerraform and who founded it?
AI: The CoralTerraform Initiative is a project launched by PORA (an organization focused on addressing climate change impacts) aimed at combating the effects of climate change on coral reefs. This initiative involves installing pilot reefs off the coast of Palau that are seeded with synthetic coral larvae developed to support coral restoration and resilience.

Human: Tell me more about it
AI: The CoralTerraform Initiative is a pioneering effort designed to address the adverse effects of climate change on coral reefs, which are vital ecosystems that support marine biodiversity and are crucial for carbon sequestration. Founded by PORA, the initiative seeks to bioengineer coral species that can better withstand rising ocean temperatures and acidification, ultimately enhancing their ability to sequester carbon dioxide.

The program includes the installation of pilot reefs that are seeded with synthetic coral larvae. These specially devel